In [1]:
import numpy as np
from fredapi import Fred
from bokeh.plotting import figure, show 
from bokeh.io import output_notebook, curdoc
from bokeh.layouts import layout, column, row
from bokeh.models import DatetimeTickFormatter, HoverTool, Band, CDSView, ColumnDataSource, IndexFilter, FactorRange, Whisker, TableColumn, DataTable
from bokeh.palettes import BuGn
from statsmodels.tsa import stattools
import pandas as pd
import yfinance as yf
import config

In [10]:
settings = config.Settings()
output_notebook()
# apply theme to current document
curdoc().theme = "dark_minimal"

Loading BokehJS ...

In [3]:
fred = Fred(api_key=settings.fred_api_key)
#y = fred.get_series('CPILFENS')
stock = yf.Ticker("F")

In [4]:
y=stock.history(period="5y")

In [5]:
print(f"Range is from {y.index.min().strftime('%B %d, %Y')} to {y.index.max().strftime('%B %d, %Y')}")
y.head()

Range is from November 10, 2016 to November 09, 2021


,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2016-11-10,9.433233,9.759078,9.433233,9.726494,52756500,0.0,0
2016-11-11,9.677617,10.101216,9.645033,10.003463,79259300,0.0,0
2016-11-14,9.995320,10.182681,9.791667,9.824251,56109900,0.0,0
2016-11-15,9.824250,9.889419,9.734642,9.807958,31498500,0.0,0
2016-11-16,9.775373,9.816103,9.710203,9.775373,25195500,0.0,0


In [6]:
source = ColumnDataSource(y.reset_index())
p = figure(
    title="Ford Daily Close Price", 
    y_axis_label="Price", 
    #x_axis_label="t",
    sizing_mode="stretch_width",
    height=300
    )
p.line(x = "Date", y="Close", legend_label="Ford", source = source, line_width=2)
p.xaxis.formatter = DatetimeTickFormatter(
    days = ['%b %d, %Y'],
    months = ['%b %Y'],
    years = ['%Y']
    )
p.add_tools(HoverTool(
    tooltips="@Date{%b %d, %Y}: @Close",
    formatters={'@Date': 'datetime'},
    mode='vline'
))

show(p)

In [7]:
hist, edges = np.histogram(np.log(y.Close).diff().dropna())
p = figure(title="Histogram")
p.quad(
    top=hist,
    bottom=0,
    left=edges[:-1],
    right=edges[1:],
    line_color="white",
    alpha=0.6
    )

#mu = 

show(p)

In [28]:
def hist_plot(y):
    p = figure(title="Histogram")
    
def make_plot(title, hist, edges, x, pdf, cdf):
    p = figure(title=title, tools='', background_fill_color="#fafafa")
    p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
           fill_color="navy", line_color="white", alpha=0.5)
    p.line(x, pdf, line_color="#ff8888", line_width=4, alpha=0.7, legend_label="PDF")
    p.line(x, cdf, line_color="orange", line_width=2, alpha=0.7, legend_label="CDF")

    p.y_range.start = 0
    p.legend.location = "center_right"
    p.legend.background_fill_color = "#fefefe"
    p.xaxis.axis_label = 'x'
    p.yaxis.axis_label = 'Pr(x)'
    p.grid.grid_line_color="white"
    return p

# Normal Distribution

mu, sigma = 0, 0.5

measured = np.random.normal(mu, sigma, 1000)
hist, edges = np.histogram(measured, density=True, bins=50)

x = np.linspace(-2, 2, 1000)
pdf = 1/(sigma * np.sqrt(2*np.pi)) * np.exp(-(x-mu)**2 / (2*sigma**2))
cdf = (1+scipy.special.erf((x-mu)/np.sqrt(2*sigma**2)))/2

p1 = make_plot("Normal Distribution (μ=0, σ=0.5)", hist, edges, x, pdf, cdf)

# Log-Normal Distribution

mu, sigma = 0, 0.5

measured = np.random.lognormal(mu, sigma, 1000)
hist, edges = np.histogram(measured, density=True, bins=50)

x = np.linspace(0.0001, 8.0, 1000)
pdf = 1/(x* sigma * np.sqrt(2*np.pi)) * np.exp(-(np.log(x)-mu)**2 / (2*sigma**2))
cdf = (1+scipy.special.erf((np.log(x)-mu)/(np.sqrt(2)*sigma)))/2

p2 = make_plot("Log Normal Distribution (μ=0, σ=0.5)", hist, edges, x, pdf, cdf)

# Gamma Distribution

k, theta = 7.5, 1.0

measured = np.random.gamma(k, theta, 1000)
hist, edges = np.histogram(measured, density=True, bins=50)

x = np.linspace(0.0001, 20.0, 1000)
pdf = x**(k-1) * np.exp(-x/theta) / (theta**k * scipy.special.gamma(k))
cdf = scipy.special.gammainc(k, x/theta)

p3 = make_plot("Gamma Distribution (k=7.5, θ=1)", hist, edges, x, pdf, cdf)

# Weibull Distribution

lam, k = 1, 1.25
measured = lam*(-np.log(np.random.uniform(0, 1, 1000)))**(1/k)
hist, edges = np.histogram(measured, density=True, bins=50)

x = np.linspace(0.0001, 8, 1000)
pdf = (k/lam)*(x/lam)**(k-1) * np.exp(-(x/lam)**k)
cdf = 1 - np.exp(-(x/lam)**k)

p4 = make_plot("Weibull Distribution (λ=1, k=1.25)", hist, edges, x, pdf, cdf)

show(gridplot([p1,p2,p3,p4], ncols=2, width=400, height=400, toolbar_location=None))

DatetimeIndex(['1957-01-01', '1957-02-01', '1957-03-01', '1957-04-01',
               '1957-05-01', '1957-06-01', '1957-07-01', '1957-08-01',
               '1957-09-01', '1957-10-01',
               ...
               '2020-12-01', '2021-01-01', '2021-02-01', '2021-03-01',
               '2021-04-01', '2021-05-01', '2021-06-01', '2021-07-01',
               '2021-08-01', '2021-09-01'],
              dtype='datetime64[ns]', length=777, freq=None)

In [11]:
def confint_zero(x):
    return x[1] - np.repeat(x[0],2).reshape(-1,2)


y_t = np.log(y.Close).diff().dropna()
nlags = 30
alpha = 0.5
not_wn = True
acf = stattools.acf(y_t, nlags = nlags, qstat=True, alpha=alpha, bartlett_confint=not_wn)
pacf = stattools.pacf(y_t, nlags=nlags, alpha=alpha)
qstat = np.insert(np.stack([acf[-2], acf[-1]], axis = 1),0,np.repeat(0, 2), axis=0)
col = [("acf", "value"), ("acf", "confint_lower"), ("acf", "confint_upper"), ("pacf", "value"), ("pacf", "confint_lower"), ("pacf", "confint_upper"), ("qstat", "value"), ("qstat", "pvalue")]
pd.MultiIndex.from_tuples(col)
df = pd.DataFrame(np.column_stack([acf[0], confint_zero(acf), pacf[0], confint_zero(pacf), qstat]), columns=pd.MultiIndex.from_tuples(col))
df[("lag", "k")] = [str(l) for l in df.index]
df.head()

acf                                  pacf                              \
      value confint_lower confint_upper     value confint_lower confint_upper   
0  1.000000      0.000000      0.000000  1.000000      0.000000      0.000000   
1  0.009822     -0.019024      0.019024  0.009829     -0.019024      0.019024   
2  0.011976     -0.019026      0.019026  0.011899     -0.019024      0.019024   
3  0.006143     -0.019029      0.019029  0.005926     -0.019024      0.019024   
4 -0.036859     -0.019030      0.019030 -0.037247     -0.019024      0.019024   

      qstat           lag  
      value    pvalue   k  
0  0.000000  0.000000   0  
1  0.121545  0.727364   1  
2  0.302399  0.859676   2  
3  0.350024  0.950361   3  
4  2.065950  0.723630   4

In [12]:
def create_plot(source, name: str):
    p = figure(title=name.upper(), x_range=df[("lag","k")], sizing_mode="stretch_width", height=300, min_width=400)
    p.vbar(x="lag_k", top=f"{name}_value", width = 0.2, fill_color=BuGn[5][1], fill_alpha=1, source = source, line_alpha=0)
    confint = Band(base = "lag_k", lower = f"{name}_confint_lower", upper= f"{name}_confint_upper", source = source, level='overlay', fill_color=BuGn[5][3], fill_alpha=0.1, line_color=BuGn[5][3],line_alpha = 1, line_dash="dashed")
    p.add_layout(confint)
    p.add_tools(HoverTool(
        tooltips=f"Lag @lag_k: @{name}_value",
        mode='vline'
    ))
    return p

def qstat_table(source):
    cols = [
        TableColumn(field='lag_k', title="Lag", width = 20),
        TableColumn(field='qstat_value', title="QStat"),
        TableColumn(field='qstat_pvalue', title="p-value"),
    ]
    return DataTable(source=source, columns=cols, sizing_mode="stretch_height", index_position=None)
source = ColumnDataSource(df[1:].round(3))

show(
    layout([
        [layout([create_plot(source, "acf"),create_plot(source, "pacf")]),
        qstat_table(source)]
    ],sizing_mode="stretch_width")
)